In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from tableone import TableOne, load_dataset


pd.options.display.max_columns = 100

In [ ]:
luisa = pd.read_csv("luisa_pts_11-2-22.csv")
luisa_list = luisa.study_id.drop_duplicates()

In [ ]:
luisa_list.shape

### medications

In [ ]:
data = pd.read_csv(
    inout.get_material_path("./02data-internal_230123_1416.csv.gz"), index_col=0)


In [ ]:
p = os.path.join(
    inout.get_path_from_settings('resources_path'),
    'edw/from_2022-09-30/patient.csv.gz'
)
#link study ID to ir_id 
patient = pd.read_csv(p)
patient = patient.rename(columns={'case_number':'patient'})
patient = patient[['patient_ir_id','patient']]

data = pd.merge(data, patient, how='left', on='patient')
p = os.path.join(
    inout.get_path_from_settings('resources_path'),
    'edw/from_2022-09-30/medication_administration.csv.gz'
)
#medication admin EDW table
meds = pd.read_csv(p, encoding= 'unicode_escape')

In [ ]:
#steroids
steroid=meds[meds['catalog_type']=='Steroid']
steroid=steroid[['patient_ir_id','base_medication_name','administration_date','administration_datetime', 'administered_dose','administered_dose_unit']]

#convert all types to hydrocortisone anti-inflammatory equivalents

def HC_equiv(row): 
    if (row['base_medication_name']=='Hydrocortisone'):
        return row['administered_dose']
    elif (row['base_medication_name']=='Dexamethasone'):
        return row['administered_dose']*25
    elif (row['base_medication_name']=='Prednisone'):
        return row['administered_dose']*4
    elif (row['base_medication_name']=='Methylprednisolone'):
        return row['administered_dose']*5
    elif (row['base_medication_name']=='Fludrocortisone'):
        return row['administered_dose']*0
    elif (row['base_medication_name']=='Prednisolone'):
        return row['administered_dose']*4
    else:
        return np.nan
steroid['HC_equiv'] = steroid.apply(HC_equiv, axis=1)

steroid2 = steroid.drop_duplicates() #multiple rxnorms for each

#sum per day
day_sum = steroid2.groupby(['patient_ir_id','administration_date']).agg({"HC_equiv": ["sum"]}) 
day_sum = day_sum.pivot_table(
    values="HC_equiv", 
    index=["patient_ir_id", "administration_date"]
).reset_index()

day_sum=day_sum.rename(columns={'sum':'hydrocort_equivalent_steroid_dose_day'})
day_sum=day_sum.rename(columns={'administration_date':'day_bucket_starts'})

#calculate pred equiv from hydrocort equiv 
day_sum['pred_equiv_per_day'] = day_sum['hydrocort_equivalent_steroid_dose_day']/5

day_sum.day_bucket_starts = pd.to_datetime(day_sum.day_bucket_starts)

In [ ]:
#join back to main dataframe 
data['day_bucket_starts']=pd.to_datetime(data['day_bucket_starts'])
data = pd.merge(data, day_sum, how='left', on=['patient_ir_id','day_bucket_starts'])

#total over admission 
admit_sum_steroid = data.groupby(['patient']).agg({"pred_equiv_per_day": ["sum"]})
admit_sum_steroid.sample(5)
admit_sum_steroid = admit_sum_steroid.pivot_table(
    values="pred_equiv_per_day", 
    index=["patient"]
).reset_index()
admit_sum_steroid=admit_sum_steroid.rename(columns={'sum':'cumulative_pred_equiv_during_admission'})

#flag
admit_sum_steroid['received_steroids_during_admission'] = admit_sum_steroid.cumulative_pred_equiv_during_admission
admit_sum_steroid.received_steroids_during_admission[admit_sum_steroid.received_steroids_during_admission != 0] = 1

#join back to main dataframe
data = pd.merge(data, admit_sum_steroid, how='left', on='patient')

In [ ]:
#il6
il6=meds[meds['catalog_type']=='IL6 Block']
il6['medication_name'] = il6['medication_name'].replace({
    'TOCILIZUMAB IVPB':'Tocilizumab',
    'STUDY: 6R88-COV-2040 SARILUMAB OR PLACEBO IVPB':'Sarilumab study',   
    'SARILUMAB IVPB':'Sarilumab'
    })
il6=il6[['patient_ir_id','medication_name','administration_date','administration_datetime', 'administered_dose','administered_dose_unit']]


il6_sum = il6.groupby(['patient_ir_id','medication_name','administration_date']).agg({"administered_dose": ["sum"]}) 

il6_sum = il6_sum.pivot_table(
    values="administered_dose", 
    index=["patient_ir_id",'medication_name', "administration_date"]
).reset_index()

il6_sum.administration_date = pd.to_datetime(il6_sum.administration_date)

il6_sum['received_tocilizumab'] = np.where(il6_sum.medication_name=='Tocilizumab',1,0)
il6_sum['received_sarilumab'] = np.where(il6_sum.medication_name=='Sarilumab',1,0)
il6_sum['sarilumab_study_drug'] = np.where(il6_sum.medication_name=='Sarilumab study',1,0)

il6_sum=il6_sum.rename(columns={'administration_date':'day_bucket_starts'})
il6_sum=il6_sum.drop(columns={'medication_name'})

#join back
data = pd.merge(data, il6_sum, how='left', on=['patient_ir_id','day_bucket_starts'])

#summarize over each admission
received_il6_during_admission = data.groupby('patient').agg(received_tocilizumab_during_admission=('received_tocilizumab', 'max'),
                       received_sarilumab_during_admission=('received_sarilumab', 'max'),
                       sarilumab_study_drug_during_admission=('sarilumab_study_drug', 'max')).reset_index()

data = pd.merge(data, received_il6_during_admission, how='left', on='patient')

In [ ]:
#remdes
remdes = meds[meds['base_medication_name']=='Remdesivir']
remdes['medication_name'] = remdes['medication_name'].replace({
    'REMDESIVIR IVPB (FROM SOLUTION)':'Remdesivir',
    'REMDESIVIR IVPB (FROM POWDER)':'Remdesivir',
    'STUDY: DMID 20-0006 REMDESIVIR IN SODIUM CHLORIDE 0.9% INJ IV':'Remdesivir study',
    'STUDY: DMID 20-0006 REMDESIVIR (GS-5734) OR PLACEBO IVPB':'Remdesivir study',
    'STUDY: GS-US-540-5821 REMDESIVIR IVPB (EXPANDED ACCESS)':'Remdesivir study',
    'STUDY: COVID19 REMDESIVIR IVPB':'Remdesivir study'  
    })
remdes=remdes[['patient_ir_id','medication_name','administration_date', 'administration_datetime', 'administered_dose']]
remdes=remdes.drop_duplicates()

remdes['received_remdesivir'] = np.where(remdes.medication_name=='Remdesivir',1,0)
remdes['remdesivir_study_drug'] = np.where(remdes.medication_name=='Remdesivir study',1,0)

remdes.administration_date = pd.to_datetime(remdes.administration_date)

remdes=remdes.rename(columns={'administration_date':'day_bucket_starts'})
remdes=remdes.drop(columns={'medication_name', 'administration_datetime', 'administered_dose'})

#join back
data = pd.merge(data, remdes, how='left', on=['patient_ir_id','day_bucket_starts'])

#summarize over each admission
received_rem_during_admission = data.groupby('patient').agg(received_remdesivir_during_admission=('received_remdesivir', 'max'),
                       remdesivir_study_drug_during_admission=('remdesivir_study_drug', 'max')).reset_index()

data = pd.merge(data, received_rem_during_admission, how='left', on='patient')

In [ ]:
data[[ 
'received_tocilizumab',
       'received_sarilumab', 'sarilumab_study_drug',
       'received_tocilizumab_during_admission',
       'received_sarilumab_during_admission',
       'sarilumab_study_drug_during_admission', 'received_remdesivir',
       'remdesivir_study_drug', 'received_remdesivir_during_admission',
       'remdesivir_study_drug_during_admission'
]] = data[[
'received_tocilizumab',
       'received_sarilumab', 'sarilumab_study_drug',
       'received_tocilizumab_during_admission',
       'received_sarilumab_during_admission',
       'sarilumab_study_drug_during_admission', 'received_remdesivir',
       'remdesivir_study_drug', 'received_remdesivir_during_admission',
       'remdesivir_study_drug_during_admission'
]].fillna(0)

### comorbidities 

In [ ]:
#comorbidities 

p = os.path.join(
    inout.get_path_from_settings('resources_path'),
    'edw/Misc_EDW_reports/6401_CCI_2022-08-16.csv'
)
#load in EDW report 6401 - recommended by Anna and Marjorie - ICD code linkage to CCI Charlson Comorbidity Index 
comorbidities = pd.read_csv(p, on_bad_lines='skip')
comorbidities['hosp_admsn_time']=pd.to_datetime(comorbidities['hosp_admsn_time'])
comorbidities=comorbidities[['ir_id','hosp_admsn_time', 'discharge_datetime',
       'icu_name', 'last_department_name', 'Myocardial_infarction',
       'Congestive_heart_failure', 'Peripheral_vascular_disease',
       'Cerebrovascular_disease', 'Dementia', 'Chronic_pulmonary_disease',
       'Rheumatic_disease', 'Peptic_ulcer_disease', 'Liver_disease',
       'Diabetes', 'Hemiplagia_or_paraplegia', 'Renal_disease', 'Cancer',
       'AIDS_HIV', 'Charlson_comorbidity_index']]

p = os.path.join(
    inout.get_path_from_settings('resources_path'),
    'edw/from_2022-09-30/basic_endpoints.csv.gz'
)

#load in basic_endpoints table for admission date for each SCRIPT study ID
endpts = pd.read_csv(p, encoding="unicode_escape")
endpts = endpts[['patient_ir_id','pt_study_id','admission_datetime']]
endpts['admission_datetime']=pd.to_datetime(endpts['admission_datetime'])

#merge on id and admisison datetime so only pull at time of admission 
endpts_comor = pd.merge(endpts, comorbidities, how='left', left_on=['patient_ir_id','admission_datetime'], right_on=['ir_id','hosp_admsn_time'])
comor_columns = ['pt_study_id','Myocardial_infarction',
       'Congestive_heart_failure', 'Peripheral_vascular_disease',
       'Cerebrovascular_disease', 'Dementia', 'Chronic_pulmonary_disease',
       'Rheumatic_disease', 'Peptic_ulcer_disease', 'Liver_disease',
       'Diabetes', 'Hemiplagia_or_paraplegia', 'Renal_disease', 'Cancer',
       'AIDS_HIV',]
endpts_comor=endpts_comor[comor_columns]
endpts_comor=endpts_comor.drop_duplicates()

#just want yes/no not weighted score
endpts_comor=endpts_comor.replace([2, 3,6], 1)
endpts_comor=endpts_comor.fillna(0)
endpts_comor=endpts_comor.rename(columns={'pt_study_id':'patient'})

### other demographics

In [ ]:
#now have mostly automatic script that MK runs to gather most of this info but this is original derivation code 

In [ ]:
# patient_category 

red = pd.read_csv("./resources/edw/from_2022-03-23-v5-data-clean/redcap_4339_pneumonia_episode_category_assessment.csv")
first_ep = red.loc[red.category_num==1, :]
first_ep=first_ep.fillna('')
first_ep['type'] = first_ep['clin_cap']+first_ep['clin_hap']+first_ep['clin_vap']
first_ep['virus']=first_ep['clin_cap_viral_npop']+first_ep['clin_hap_viral_npop']+first_ep['clin_vap_viral_npop']
new_dem = first_ep[['pt_study_id','pt_category','virus','type']]
def category(row): 
    if (row['pt_category'].strip()=='Non-pneumonia control'):
        return 'Non-pneumonia control'
    elif (('SARS-Cov-2') in row['virus']):
        return 'COVID-19'
    elif (row['type'].strip()=='Viral/Etiology defined'):
        return 'Other Viral Pneumonia'
    elif (row['type'].strip()=='Bacterial/viral co-infection'):
        return 'Other Viral Pneumonia'    
    else:
        return 'Other Pneumonia'
new_dem['Patient_category'] = new_dem.apply(category, axis=1)
to_drop = ['pt_category', 'virus', 'type',]
new_dem=new_dem.drop(columns=to_drop)
new_dem['COVID_status']=np.where(new_dem.Patient_category=='COVID-19',True,False)


In [ ]:
# Basic endpoints 
basic_endpts = pd.read_csv("./resources/edw/from_2022-03-23-v5-data-clean/basic_endpoints.csv.gz", encoding="unicode_escape")
endpoints_cols = """
pt_study_id
patient_ir_id
admission_datetime
death_date
discharge_disposition_name
discharge_datetime
external_transfer_flag
admission_source_name
total_icu_los_days
tracheostomy_flag
ecmo_yn
lung_transplant_date
First_intub_start
First_intub_stop
Second_intub_start
Second_intub_stop
Third_intub_start
Third_intub_stop
Fourth_intub_start
Fourth_intub_stop
Fifth_intub_start
Fifth_intub_stop
""".strip().split("\n")
basic_endpts = basic_endpts[endpoints_cols]
basic_endpts.sample(5)

# calculate cumulative intubation duration
#find intubation date columns
dates = [
    'First_intub_start', 'First_intub_stop',
       'Second_intub_start', 'Second_intub_stop', 'Third_intub_start',
       'Third_intub_stop', 'Fourth_intub_start', 'Fourth_intub_stop',
       'Fifth_intub_start', 'Fifth_intub_stop'
]
#turn into dates
basic_endpts[dates] = basic_endpts[dates].apply(pd.to_datetime)
#calculate durations
basic_endpts['first_int_duration'] = basic_endpts['First_intub_stop']-basic_endpts['First_intub_start']
basic_endpts['second_int_duration'] = basic_endpts['Second_intub_stop']-basic_endpts['Second_intub_start']
basic_endpts['third_int_duration'] = basic_endpts['Third_intub_stop']-basic_endpts['Third_intub_start']
basic_endpts['fourth_int_duration'] = basic_endpts['Fourth_intub_stop']-basic_endpts['Fourth_intub_start']
basic_endpts['fifth_int_duration'] = basic_endpts['Fifth_intub_stop']-basic_endpts['Fifth_intub_start']
#fill 0s for NAs
basic_endpts['second_int_duration'] = basic_endpts['second_int_duration'].fillna(pd.Timedelta(seconds=0))
basic_endpts['third_int_duration'] = basic_endpts['third_int_duration'].fillna(pd.Timedelta(seconds=0))
basic_endpts['fourth_int_duration'] = basic_endpts['fourth_int_duration'].fillna(pd.Timedelta(seconds=0))
basic_endpts['fifth_int_duration'] = basic_endpts['fifth_int_duration'].fillna(pd.Timedelta(seconds=0))
#sum up durations 
basic_endpts['Cumulative_intubation_days']=basic_endpts['first_int_duration']+basic_endpts['second_int_duration']+basic_endpts['third_int_duration']+basic_endpts['fourth_int_duration']+basic_endpts['fifth_int_duration']

#drop columns used to calculate
basic_endpts=basic_endpts.drop(columns=[
        'First_intub_start', 'First_intub_stop',
       'Second_intub_start', 'Second_intub_stop', 'Third_intub_start',
       'Third_intub_stop', 'Fourth_intub_start', 'Fourth_intub_stop',
       'Fifth_intub_start', 'Fifth_intub_stop',
    'first_int_duration','second_int_duration','third_int_duration','fourth_int_duration','fifth_int_duration'
])

renames = {
    "Home with Home Health Care": "Home", 
    "Home or Self Care": "Home",
    "Against Medical Advice (AMA) or Elopement": "Home", 
    "Group Home": "Home", 
    "Home with Equipment or O2": "Home", 
    "Home with Outpatient Services": "Home", 
    "Inpatient Psychiatric Hospital": "Home",
        
    "Acute Inpatient Rehab": "Rehab",
    "Planned Readmission – DC/transferred to acute inpatient rehab": "Rehab",
    "Acute Inpatient Rehabilitation": "Rehab",
    
    "Skilled Nursing Facility or Subacute Rehab Care":"SNF",
    
    "Acute Care Hospital": "SNF", #manually reviewed this weird case and was supposed to d/c to SNF 
    "Long-Term Acute Care Hospital (LTAC)": "LTACH", 
     
    "Home with Hospice": "Hospice", 
    "Inpatient Hospice": "Hospice", 

    "Expired": "Died"
}
basic_endpts.discharge_disposition_name.replace(renames, inplace=True)
basic_endpts["Binary_outcome"] = 0
basic_endpts.loc[basic_endpts.discharge_disposition_name.isin(["Hospice", "Died"]), "Binary_outcome"] = 1


In [ ]:
p = os.path.join(
    inout.get_path_from_settings('resources_path'),
    'edw/from_2022-09-30/redcap_4339_demographics.csv.gz'
)
#add immunocomp status from redcap demographics - filled in by SCRIPT research team

red = pd.read_csv(p)

red = red[[ 'pt_study_id','pt_immunocomp', 'type_immunocomp','organ_transplant', ]]
red=red.rename(columns={'pt_study_id':'patient'})
red=red.rename(columns={'pt_immunocomp':'Imuunocompromised_flag'})

endpts_comor = pd.merge(endpts_comor,red, how='left', on='patient')
dem = pd.merge(dem, endpts_comor, how='left', on='patient')

In [ ]:
#rename columns 

dem.rename(columns={'Imuunocompromised_flag':'Immunocompromised_flag', 'cumulative_pred_equiv_during_admission':'Cumulative prednisone equivalents over admission'})

In [ ]:
patient = pd.read_csv("./resources/edw/from_2022-09-30/patient.csv.gz")
patient = patient[['case_number','cohort_patient_id','patient_ir_id']]
patient_identifiers = pd.read_csv("./resources/edw/from_2022-09-30/redcap_4339_patient_identifiers.csv.gz")
age = patient_identifiers[['pt_study_id', 'pt_age', 'initial_bal_dt', ]]
person = pd.read_csv("./resources/edw/from_2022-03-23-v5-data-clean/person.csv.gz")
race_ethnicity = person[['person_id','race_concept_name','ethnicity_concept_name','gender_concept_name']]
bmi = pd.read_csv("SCRIPT BMI data 11-14-22.csv")

In [ ]:
patient_age = pd.merge(patient, age, how='left', left_on='case_number', right_on='pt_study_id')
patient_age_race = pd.merge(patient_age, race_ethnicity, how='left', left_on='cohort_patient_id', right_on='person_id')
patient_age_race_cat = pd.merge(patient_age_race, new_dem, how='left', on='pt_study_id')
patient_age_race_cat_endpts = pd.merge(patient_age_race_cat,basic_endpts, how='left', on='pt_study_id')
patient_age_race_cat_endpts_bmi = pd.merge(patient_age_race_cat_endpts, bmi, how='left', left_on='patient_ir_id_x', right_on='ir_id')


In [ ]:
# admission SOFA score

sofa = pd.read_csv("./resources/edw/from_2022-09-30/sofa_scores.csv.gz")
first_two_sofa = sofa.groupby('pt_study_id').head(2)
#fill NA with 0 and re-sum
first_two_sofa[['P_F_ratio_points', 'platelet_points','bilirubin_points', 'gcs_points', 'renal_points', 'htn_points']] = first_two_sofa[['P_F_ratio_points', 'platelet_points', 'bilirubin_points', 'gcs_points', 'renal_points', 'htn_points']].fillna(0)
col_list= ['P_F_ratio_points', 'platelet_points','bilirubin_points', 'gcs_points', 'renal_points', 'htn_points']
first_two_sofa['SOFA_resum'] = first_two_sofa[col_list].sum(axis=1)
#group
worst2sofa = first_two_sofa[['pt_study_id','SOFA_resum']].groupby('pt_study_id').max().reset_index()
worst2sofa=worst2sofa.rename(columns={'SOFA_resum':'Admit_SOFA_score'})

# admission apache score from EDW report

aps = pd.read_csv("./SCRIPT Serial APS Score 6-15-22.csv")
first_two=aps.groupby('pt_study_id').head(2)
worst2aps = first_two[['pt_study_id','total_APS_score_num']].groupby('pt_study_id').max().reset_index()
worst2aps = worst2aps.rename(columns={'total_APS_score_num':'Admit_APS_score'})


In [ ]:
admission_severity = pd.merge(worst2sofa, worst2aps, how='left', on='pt_study_id')
patient_age_race_cat_endpts_bmi_comor_sofa = pd.merge(patient_age_race_cat_endpts_bmi_comor, admission_severity, how='left', left_on='case_number', right_on='pt_study_id')
patient_age_race_cat_endpts_bmi_comor_sofa = patient_age_race_cat_endpts_bmi_comor_sofa.rename(columns={
    'race_concept_name':'Race',
    'ethnicity_concept_name':'Ethnicity',
    'gender_concept_name':'Gender',
    'death_date': 'Death_date',
    'discharge_disposition_name': 'Discharge_disposition',
    'external_transfer_flag': 'External_transfer_flag',
    'admission_source_name': 'Admission_source_name',
    'binary_outcome': 'Binary_outcome',
    'total_icu_los_days':'Cumulative_ICU_days',
       'tracheostomy_flag':'Tracheostomy_flag',
    'pt_age':'Age',
    
    })

In [ ]:
parameters_order = [
    'patient', 'External_transfer_flag', 'Admission_source_name',
    'Discharge_disposition', 'Binary_outcome', 'Death_date',
    'Patient_category', 'COVID_status',
    'Age', 'Gender', 'Ethnicity','Race', 'BMI', 'Admit_SOFA_score', 'Admit_APS_score',
    'Cumulative_ICU_days','Cumulative_intubation_days','Tracheostomy_flag',
    #comorbidities 
    'Myocardial_infarction',
       'Congestive_heart_failure', 'Peripheral_vascular_disease',
       'Cerebrovascular_disease', 'Dementia', 'Chronic_pulmonary_disease',
       'Rheumatic_disease', 'Peptic_ulcer_disease', 'Liver_disease',
       'Diabetes', 'Hemiplagia_or_paraplegia', 'Renal_disease', 'Cancer',
       'Imuunocompromised_flag',
]
patient_age_race_cat_endpts_bmi_comor_sofa_clean = patient_age_race_cat_endpts_bmi_comor_sofa[parameters_order]

In [ ]:
patient_age_race_cat_endpts_bmi_comor_sofa_clean.to_csv("patient_age_race_cat_endpts_bmi_comor_sofa_clean_11-15-22.csv")

# filter on just Luisa's patients

In [ ]:
#filter on just Luisa's patients
cd = data.copy()
cd_luisa = cd[cd.patient.isin(luisa_list)]

In [ ]:
mytable = TableOne(cd_luisa, columns=['Age', 'Ethnicity', 'Gender', 'Race', 'Smoking_status',
       'BMI', 'Admit_APS_score','Admit_SOFA_score', 'Cumulative_ICU_days',
       'Number_of_ICU_stays', 'Tracheostomy_flag',
       'Cumulative_intubation_days','Discharge_disposition',
                                 'Cumulative prednisone equivalents over admission',
 'received_steroids_during_admission',
 
 'received_tocilizumab_during_admission',
 'received_sarilumab_during_admission',
 'sarilumab_study_drug_during_admission',
 
 'received_remdesivir_during_admission',
 'remdesivir_study_drug_during_admission',
                                'Congestive_heart_failure',
 'Peripheral_vascular_disease',
 'Cerebrovascular_disease',
 'Chronic_pulmonary_disease',
 'Peptic_ulcer_disease',
 'Liver_disease',
 'Diabetes',
 'Renal_disease',
 'Cancer',
 'Immunocompromised_flag',
                                ], categorical=['Ethnicity', 'Gender', 'Race', 'Smoking_status',
       'Tracheostomy_flag','Discharge_disposition',
                                                'received_steroids_during_admission',
 
 'received_tocilizumab_during_admission',
 'received_sarilumab_during_admission',
 'sarilumab_study_drug_during_admission',
 
 'received_remdesivir_during_admission',
 'remdesivir_study_drug_during_admission',
                                             'Congestive_heart_failure',
 'Peripheral_vascular_disease',
 'Cerebrovascular_disease',
 'Chronic_pulmonary_disease',
 'Peptic_ulcer_disease',
 'Liver_disease',
 'Diabetes',
 'Renal_disease',
 'Cancer',
 'Immunocompromised_flag',  ], nonnormal=['Age',  'BMI', 'Admit_APS_score','Admit_SOFA_score', 'Cumulative_ICU_days',
       'Number_of_ICU_stays',  'Cumulative_intubation_days','Cumulative prednisone equivalents over admission',
                                                            ],
                   groupby='Patient_category')
mytable

### labs on day of BAL

In [ ]:
labs = pd.read_csv("/Users/catgaohow/OneDrive - Northwestern University/cereal/materials/CAG/01_assembling_data_full/01_dataframe_full_only_labs.csv.gz", index_col=0)
labs_date = pd.merge(labs, initial, how='left', on='pt_study_id')
labs_date.initial_bal_dt=pd.to_datetime(labs_date.initial_bal_dt)
labs_date.day_bucket_starts=pd.to_datetime(labs_date.day_bucket_starts)
labs_date['day_of_enrollment'] = np.where((labs_date['day_bucket_starts']==labs_date['initial_bal_dt']),1,0)
data_enroll_bal=labs_date[labs_date['day_of_enrollment']==1]
data_enroll_bal_luisa = data_enroll_bal[data_enroll_bal.pt_study_id.isin(luisa_list)]
data_enroll_bal_luisa=data_enroll_bal_luisa.drop_duplicates()
labs_dayofenrollmentbal_luisa = pd.merge(data_enroll_bal_luisa, luisa_category, how='left', left_on='pt_study_id', right_on='patient')
labs_dayofenrollmentbal_luisa=labs_dayofenrollmentbal_luisa[['pt_study_id', 'Patient_category', 'initial_bal_dt',
                                                             'date_y', 'Albumin', 'Bicarbonate', 'CRP', 'D-dimer',
       'Ferritin', 'Hemoglobin', 'LDH', 'Lactic Acid', 'Lymphocytes',
       'Neutrophils', 'Procalcitonin', 'WBC count',
]]
labs_dayofenrollmentbal_luisa.to_csv('Luisa_labs_dayofenrollmentbal_11-16-22.csv')


In [ ]:
mytable = TableOne(labs_dayofenrollmentbal_luisa, columns=[
       'Albumin', 'Bicarbonate', 'CRP', 'D-dimer',
       'Ferritin', 'Hemoglobin', 'LDH', 'Lactic Acid', 'Lymphocytes',
       'Neutrophils', 'Procalcitonin', 'WBC count',
], 
                   categorical=[
                   
                   ], nonnormal=['Albumin', 'Bicarbonate', 'CRP', 'D-dimer',
       'Ferritin', 'Hemoglobin', 'LDH', 'Lactic Acid', 'Lymphocytes',
       'Neutrophils', 'Procalcitonin', 'WBC count',] ,
                   groupby='Patient_category')
mytable

In [ ]:
mytable.to_csv("Luisa_labs_dayofenrollmentbal_table_11-16-22.csv")